In [1]:
import timm
import torch
import wandb
import fastai
from fastai.callback.wandb import WandbCallback
from fastai.vision.all import *
from fastai.vision.core import *
from fastai.text.core import RegexLabeller
from fastai.vision.utils import get_image_files
from fastai.data.block import DataBlock
from fastai.data.core import *

In [2]:
# config = SimpleNamespace(
#     batch_size=2,
#     img_size=224,
#     seed=42,
#     pretrained=True,
#     model_name="maxvit_rmlp_small_rw_224.sw_in1k", # try with maxvit_nano_rw_256.sw_in1k # regnetx_040 coatnet_bn_0_rw_224.sw_in1k
#     epochs=3)

# def get_planets(batch_size=8, img_size=224, seed=42):
#     "A sample of the planets dataset"
#     # dataset_path=untar_data(URLs.PLANET_SAMPLE)
#     dls = ImageDataLoaders.from_csv(dataset_path, 
#                                     folder="train", 
#                                     csv_fname="labels.csv",
#                                     label_delim=" ",
#                                     suff=".jpg",
#                                     bs=batch_size,
#                                     seed=seed,
#                                     item_tfms=Resize(img_size))
#     return dls

# def train(config):
#     "Train the model using the supplied config"
#     dls = get_planets(config.batch_size, config.img_size, config.seed)
#     with wandb.init(project="PROJECT", group='ambrosia_symbiosis', job_type='test_training', config=config):
#         cbs = [MixedPrecision(), WandbCallback(log='all')]
#         learn = vision_learner(dls, config.model_name, metrics=[accuracy_multi], #error_rate, 
#                                cbs=cbs, pretrained=config.pretrained)
#         # learn.remove_cb(ProgressCallback)
#         learn.fine_tune(config.epochs)

In [3]:
config = SimpleNamespace(
    batch_size=8,
    img_size=224,
    seed=42,
    pretrained=True,
    model_name="maxvit_rmlp_small_rw_224.sw_in1k", # try with maxvit_nano_rw_256.sw_in1k # regnetx_040 coatnet_bn_0_rw_224.sw_in1k
    epochs=3)

def get_images(dataset_path, batch_size, img_size, seed):
    "The beetles dataset"
    files = get_image_files(path=dataset_path, recurse=True, folders=('train','valid'))
    # dls = ImageDataLoaders.from_name_re(dataset_path, files, 
    #                                     pat=r'(^[a-zA-Z]+_*[a-zA-Z]+)', 
    #                                     valid_pct=0.2, 
    #                                     seed=seed, 
    #                                     bs=batch_size,
    #                                     item_tfms=Resize(img_size, ResizeMethod.Pad, pad_mode='zeros'))
    
    dblock = DataBlock(blocks = (ImageBlock, CategoryBlock),
                       get_items = get_image_files,
                       splitter = GrandparentSplitter(train_name='train', valid_name='valid'),
                       get_y = parent_label,
                       # get_y = RegexLabeller(pat = r'([a-zA-Z]+_[a-zA-Z]+)(?=[^\/]+$)'),
                       item_tfms = Resize(img_size, ResizeMethod.Pad, pad_mode='zeros'))
    dls = dblock.dataloaders(dataset_path, bs = batch_size) # [os.path.join(dataset_path,'train'), os.path.join(dataset_path,'valid')]
    return dls

def train(config, dataset_path):
    "Train the model using the supplied config"
    dls = get_images(dataset_path=dataset_path, batch_size=config.batch_size, img_size=config.img_size, seed=config.seed)
    with wandb.init(project="PROJECT", group='ambrosia_symbiosis', job_type='test_training', config=config):
        cbs = [MixedPrecision(), WandbCallback(log='all')]
        learn = vision_learner(dls, config.model_name, metrics=[error_rate, 
                                                                accuracy, 
                                                                top_k_accuracy, 
                                                                ##BrierScore,
                                                                ##CohenKappa,
                                                                ##F1Score,
                                                                FBeta,
                                                                # HammingLoss,
                                                                # Jaccard,
                                                                # Precision,
                                                                # Recall,
                                                                # RocAuc,
                                                                # MatthewsCorrCoef,
                                                               ], 
                               cbs=cbs, pretrained=config.pretrained)
        learn.fine_tune(config.epochs)

In [4]:
train(config=config, dataset_path=r"F:\selected_images\train_data")

epoch,train_loss,valid_loss,error_rate,accuracy,top_k_accuracy,FBeta,time


epoch,train_loss,valid_loss,error_rate,accuracy,top_k_accuracy,FBeta,time


epoch,train_loss,valid_loss,error_rate,accuracy,top_k_accuracy,FBeta,time


epoch,train_loss,valid_loss,error_rate,accuracy,top_k_accuracy,FBeta,time


epoch,train_loss,valid_loss,error_rate,accuracy,top_k_accuracy,FBeta,time


...

epoch,train_loss,valid_loss,error_rate,accuracy,top_k_accuracy,FBeta,time


epoch,train_loss,valid_loss,error_rate,accuracy,top_k_accuracy,FBeta,time


epoch,train_loss,valid_loss,error_rate,accuracy,top_k_accuracy,FBeta,time


epoch,train_loss,valid_loss,error_rate,accuracy,top_k_accuracy,FBeta,time


epoch,train_loss,valid_loss,error_rate,accuracy,top_k_accuracy,FBeta,time
